Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

$\textbf{Answer}$

Say V is the event that the defendent is guilty (1) or innocent (0), and that E is the event that the defendent has the blood type (1) or not (0). Prosecutor's fallacy falsely claims that $P(E|V) = P(V|E)$. When actually, $P(E|V) = \frac{P(V|E)P(E)}{P(V)}$. In other words, the fallacy ignores the probability of the defendent being guilty independent of the blood evidence, $P(V)$, and the probability of having the rare blood type independent of the verdict, $P(E)$. The former being an issue because it ignores all other evidence, while the latter is important to consider the magnitude having the rare blood at all compared to having the rare blood only if guilty.

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument (HINT: What happens to the prior in this case if there is *other* evidence presented)?

$\textbf{Answer}$

If there is an established $P(V=1)$ from the other evidence, presumably the evidence has already influenced $P(V=1)$ (possibly very far) away from the notion that $\textit{anyone}$ in the city is guilty. The probability of an unlikely event matching one suspect is highly relevant when talking about a subset of the people in the city rather than the whole group at large, since it is less likely for a random group of 10, for example, to have someone who happens to have the blood type. Regardless, even with no prior evidence the blood test would still be "relevant" because it would narrow the prior from 1/800000 to 1/8000.

c. Suppose that forensic analysis tells us that that the blood test has 98% sensitivity (true positive rate) and a 1% false positive rate.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* that there is no other evidence.  

$\textbf{Answer}$

V is the event the defendent is guilty.
E is the event the defendent has the blood type.
T is the event the defendent has a positive blood test.

Given statistics:
$\hspace{5mm}P(V)=0.2\hspace{10mm}P(E)=0.01\hspace{10mm}P(T|E)=0.98\hspace{10mm}P(T|!E)=0.01$

Bayes' Theorem:
$\hspace{5mm}P(V|T) = \frac{P(T|V)P(V)}{P(T)}$

We can marginalize the denominator for all positive test result possibilities (if the person is guilty or innocent).

$P(V|T) = \frac{(P(T|V)P(V)}{P(T|V)P(V)+P(T|!V)P(!V)}$

We're told that the person that is guilty and the defendent are guaranteed to have the same blood type so $P(T|V)=P(T|E)=0.98$ and similarly $P(T|!V)=P(T|!E)=0.01$. Substituting values:

$P(V|T) = \frac{(0.98)(0.2)}{(0.98)(0.2)+(0.01)(0.8)}\approx0.961$

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [24]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [78]:
#What is the prior P(Y=ham) ?
ham_prior = np.sum(training_labels)/len(training_labels) #!

print('ham_prior:', ham_prior, '\n')

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    total_of_key = val[0]+val[1]
    ham_count = val[1]
    ham_likelihood[key] = ham_count/total_of_key #!
    
# Example of ham likelihoods
import itertools
dict(itertools.islice(ham_likelihood.items(), 10))

ham_prior: 0.8701793721973095 



{'no': 0.875,
 "i'm": 0.980327868852459,
 'good': 0.9272727272727272,
 'for': 0.7145454545454546,
 'the': 0.8442545109211775,
 'movie,': 0.6666666666666666,
 'is': 0.8236130867709816,
 'it': 0.9447368421052632,
 'ok': 0.9621212121212122,
 'if': 0.9169329073482428}

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [80]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = ham_prior 
    posterior_spam = 1-ham_prior
    # Loop over all the words in each message
    for w in m:
        #! What is the purpose of this try/except handler?
        # Skip words not in the training set 
        try:
            posterior_ham *=  ham_likelihood[w] #!
            posterior_spam *= (1-ham_likelihood[w]) #!
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)
    
for i in range(10):
    print(posteriors[i])

[1.26095525e-14 1.00000000e+00]
[2.65423069e-07 9.99999735e-01]
[9.99999996e-01 3.80127070e-09]
[1.90853281e-13 1.00000000e+00]
[4.51555027e-17 1.00000000e+00]
[3.06572028e-14 1.00000000e+00]
[9.74875935e-15 1.00000000e+00]
[3.1763994e-11 1.0000000e+00]
[1.00000000e+00 1.42682097e-10]
[9.99478517e-01 5.21483203e-04]


Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [81]:
correct = 0

for i in range(len(test_labels)):
    predict = 2
    if (posteriors[i,0] > posteriors[i,1]):
        predict = 0
    else:
        predict = 1
    if (test_labels[i] == predict):
        correct += 1

print('Accuracy: %.3f%%' % (correct/len(test_labels)*100))

Accuracy: 93.896%
